In [ ]:
'''This part of the code is the original experimental code of T-kernel SGD in Subsection 5.1. Different data distribution and noise can be set by 
adjusting the fuction "construct-data", as well as the expression after function "grho_cal"'s return to set different optimal prediction functions.'''

In [10]:
#coding=gbk
import math
import random
import matplotlib.pyplot as plt
import time
import numpy as np
from decimal import *
getcontext().prec=36

class T_kernel_SGD():
    def __init__(self,d,s):
        self.d = d
        self.s = s
        self.Tn = [np.array([1.])]
        self.K_n = [[np.array([0.])]]
        self.K_t = [[ np.array([1.]) ]]
        self.L_n = 0
        self.fn = [[np.array([0.])]]
        self.f_average_n = [[np.array([0.])]]
        self.computing_ploy = []
        self.gamma0 = 1 #Initial step size
        self.t = 0 #Step decay rate
        self.epoch = 1 #  Current number of iterations
        self.theta = 1/(1+2*self.s)
        self.dim_Pi_Ln_d = 1
        self.cc_B4 = -math.pi**4/48
        
    def factorial(self,a,b,i,model):#i是间隔的步长,model是输出模式，输出列表或值
        if model == 'list':
            S = 1
            Output = []
            for k in range(0,b,i):
                S *= (k+a)
                Output.append(S)
        elif model ==  'value':
            S = 1
            for k in range(0,b,i):
                S *= (k+a)
            Output = S
        return Output
    
    def jiecheng(self,x):
        S = 1
        for i in range(1,x+1):
            S *= i
        return S
    
    def jiecheng_list(self,x):
        S = 1
        Output = [1]
        for i in range(1,x+1):
            S *= i
            Output.append(S)
        return Output
            
            
    
    def combination_num(self,a,b):#C^a_b
        if a == -1:
            C = 0
        elif a == 0:
            C = 1
        else:
            C = self.jiecheng(b)//(self.jiecheng(a)*self.jiecheng(b-a))
        return C
    
    def Multivariate_binomial(self,d,k,i):
        #self.T_n  is a polynomial before
        #k is the order of polynomial which will be calculating
        #d is dimension i in the element
        self.T_L_n = []
        if k == 1:
            self.T_L_n = np.array([1.])
        else:
            for a in self.Tn[i:]:
                self.T_L_n = np.append(self.T_L_n,a)
            Dim_list = [0]+[self.combination_num(l,d+l-1) for l in range(k)]
            for l in range(1,len(Dim_list)):
                for p in range(Dim_list[l-1],Dim_list[l]):
                    self.T_L_n[p] = self.T_L_n[p]*(k/(k-l+1))
        
        
    def updating_Zt(self):
        self.K_n = [[np.array([0.])]]
        if self.L_n >= 1:
            self.Tn = [np.array([1.])]
            omega_Ln = (self.dim_Pi_Ln_d)**(-2*self.s)
            fact1 = self.factorial(self.d/2,self.L_n,1,'value')#(d/2)k
            fact2 = [1]+self.factorial(2-self.L_n-self.d/2,int(self.L_n/2),1,'list')#(2-k-d/2)j
            fact3 = self.jiecheng_list(int(self.L_n/2))#(j)!
            fact4 = self.jiecheng_list(self.L_n)#(k-2j)!
            fact5 = [2**l for l in range(self.L_n+1)]
            #print("update Ln","1,",fact1,fact2,"3",fact3,fact4,self.L_n)
            Coe = [0 for l in range(self.L_n+1)]
            if self.L_n%2 == 0:
                for i in range(self.L_n+1):
                    if i%2 == 0:
                        Coe[i] = omega_Ln*fact1*fact5[i]/(fact2[-i//2-1]*fact3[-i//2-1]*fact4[i])
                    else:
                        Coe[i] = 0
            elif self.L_n%2 == 1:
                for i in range(self.L_n+1):
                    if i%2 == 1:
                        Coe[i] = omega_Ln*fact1*fact5[i]/(fact2[-(i+1)//2]*fact3[-(i+1)//2]*fact4[i])
                    else:
                        Coe[i] = 0
            #print(omega_Ln)
            #print(self.L_n,"Coe",Coe)            
            self.K_n[0][0][0] += Coe[0]*1   
            for k in range(1,self.L_n+1):
                self.Tn_lin = []
                for i in range(self.d):
                    self.Multivariate_binomial(self.d-i,k,i)
                    self.Tn_lin.append(self.T_L_n[:])
                self.Tn = self.Tn_lin[:]
                #print("L_n",self.L_n,k,self.Tn)
                self.K_n.append([ Coe[k]*A for A in self.Tn])
            self.K_t.append([np.zeros(len(A)) for A in self.Tn])
            self.K_t = [[a+b for a,b in zip(A,B)] for A,B in zip(self.K_t,self.K_n)]
            
    def Computation_ploynomial(self,x,order):#Note that here order is the order of the polynomial
        self.Tn = [np.array([1.])]
        if order == 0:
            self.computing_ploy = [self.Tn[:]]
        if order >= 1:
            self.computing_ploy = [self.Tn[:]]
            self.Tn = [ np.array([x[i]]) for i in range(self.d)]
            self.computing_ploy.append(self.Tn[:])
            for i in range(2,order+1):
                self.Tn_lin = []
                B = []
                for j in range(self.d-1,-1,-1):
                    B = np.append(self.Tn[j],B)
                    self.Tn_lin.insert(0,x[j]*B)
                self.Tn = self.Tn_lin[:]
                self.computing_ploy.append(self.Tn[:])
            
    def Computation_K_Xnt(self,X):
        self.Computation_ploynomial(X,self.L_n)
        self.K_Xnt_list = [[ a*b for a,b in zip(A,B)] for A,B in zip(self.computing_ploy,self.K_t)]     
    
    def Computation_fn(self,X):
        self.Computation_ploynomial(X,self.L_n)
        return sum([sum([ np.dot(a,b) for a,b in zip(A,B)])for A,B in zip(self.computing_ploy,self.fn)])
    
    def Computation_fn_average(self,X):
        self.Computation_ploynomial(X,self.L_n)
        return sum([sum([ np.dot(a,b) for a,b in zip(A,B)])for A,B in zip(self.computing_ploy,self.f_average_n)])
    
    def iteration(self,X,Y):
        coe = self.gamma0*((self.epoch)**(-self.t))*(Y - self.Computation_fn(X))
        if self.dim_Pi_Ln_d+1 < self.epoch**self.theta:
            self.L_n += 1 
            self.dim_Pi_Ln_d = self.combination_num(self.L_n,self.L_n+self.d-1)+self.combination_num(self.L_n-1,self.L_n+self.d-2)
            self.updating_Zt()
            A = [np.zeros(len(B)) for B in self.Tn]
            self.fn.append(A[:])
            self.f_average_n.append(A[:])
        self.Computation_K_Xnt(X)
        self.fn= [[ a+coe*b for a,b in zip(A,B)] for A,B in zip(self.fn,self.K_Xnt_list)] 
        self.f_average_n = [[ (self.epoch*a+b)/(self.epoch+1) for a,b in zip(A,B)] for A,B in zip(self.f_average_n,self.fn)]
    """    
    def construct_data(self):
        X = [random.gauss(0,1) for i in range(self.d)]
        mo = np.sqrt(sum([i**2 for i in X]))
        X = [i/mo for i in X]
        Y = np.sin(sum(X))
        Z = Y+random.gauss(0,0.05)
        return [X,Z,Y]
        """
    '''
    def construct_data(self):
        X = [random.gauss(0,1) for i in range(self.d)]
        mo = np.sqrt(sum([i**2 for i in X]))
        X = [i/mo for i in X]
        Y = 10*np.sin(X[0]*X[1])+20*(X[2]-0.5)**2+10*X[3]+5*X[4]+X[1]*X[2]*X[3]+np.cos(X[2]*X[3])
        Z = Y+random.gauss(0,0.2)
        return [X,Z,Y]
    '''
    def grho_cal(self,theta):
        t = theta/(2*math.pi)
        return  (t**4-2*(t**3)+t**2-1/30)#(t**2-t+1/6)

    def construct_data(self):
        theta = 2*math.pi*random.random()
        Y = self.grho_cal(theta)
        Z = Y + random.uniform(-0.2,0.2)
        return [[np.cos(theta),np.sin(theta)],Z,Y]

    def test(self):
        S1 = 0
        S2 = 0
        end = time.perf_counter()
        for j in range(1000):
            Z = self.construct_data()
            S1 +=(self.Computation_fn(Z[0])-Z[-1])**2
            S2 +=(self.Computation_fn_average(Z[0])-Z[-1])**2
        #print('iteration:',i,S1/1000,S2/1000,self.L_n,end-start)
        return [S1/1000,S2/1000]
    
    def updating(self,gamma0,t,theta,Sum_epoch,NN,Uni):
        self.gamma0 = gamma0
        self.t = t
        self.theta = theta
        self.Error = []
        self.Time = []
        start = time.perf_counter()
        for i in range(1,Sum_epoch+1):
            self.epoch = i
            Z = self.construct_data()
            self.iteration(Z[0],Z[-2])
            if i in NN:
                if Uni == 'time':
                    end = time.perf_counter()
                    self.Time.append(end-start)
                    print(i,end-start)
                elif Uni == 'error':
                    S1,S2= self.test()
                    self.Error.append(S2)
                    print(i,S2)
                #print(i,end-start)
                #self.Error.append(S2)
                #print(self.f_average_n)
                #print('K_n',self.K_n)
         

In [11]:
d = 2
N = [int(10**(i/5))for i in range(1,40)]
s = 1.
r=3/4
gamma0 =0.3
t = 0.0
theta = 1/(4*r*s+1)
ACC_B2 = []
Sum_epoch = int(1*10**(6)+1)
T = 20
t_kernel_SGD = T_kernel_SGD(d,s)
t_kernel_SGD.updating(gamma0,t,theta,Sum_epoch,N,'error')


1 0.0007686230024191391
2 0.0009796175777620837
3 0.0005631828572312699
6 0.0005374426950103163
10 0.0005006187699631088
15 0.00047857164569840816
25 0.0007997553265583749
39 0.0008870942428038156
63 0.0012600850738374732
100 0.000466007987479568
158 0.0004980040293255678
251 0.0003330850901560405
398 0.0001451801632108018
630 0.0001228695165645903
1000 4.6075672656278275e-05
1584 8.524765877263309e-06
2511 2.220888241551208e-05
3981 1.3203023124167195e-05
6309 1.1894548023285223e-05
10000 1.14902016126687e-05
15848 6.544392642103599e-06
25118 4.916373631464588e-06
39810 4.225569836175325e-06


KeyboardInterrupt: 